<a href="https://colab.research.google.com/github/DejiangZ/WildPPG-DataEDA/blob/master/Lab07_Repeated_Measures_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab #7. Repeated Measures Analysis for Sensor & Human Data**
In this lecture, we will explore how to choose the right method for analyzing repeated measures data.

*This material is a joint work of TAs from IC Lab at KAIST, including Gyuna Kim, Dejiang Zhang, and Panyu Zhang. This work is licensed under CC BY-SA 4.0.*

### Preparation
We will proceed with installing Python libraries and data

In [ ]:
!pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 3.4 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/gn0219/IoTLab07.git

Cloning into 'IoTLab07'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 53.04 KiB | 3.79 MiB/s, done.
Resolving deltas: 100% (5/5), done.


# Overview: What We'll Learn Today

In this session, we’ll explore several statistical methods for analyzing **repeated measures data** — data where multiple measurements are collected from the same participants over time.

Many traditional statistical methods like the independent t-test assume that all observations are independent. However, in repeated measures designs, this assumption is violated due to **correlation within or between subjects**.  
Applying methods that ignore this correlation can lead to **misestimated standard errors** and ultimately **invalid test results**.

## What You'll Learn Today

1. **What is repeated measures data?**
   - Why the assumption of independence is violated
   - Why repeated measures designs are useful

2. **How to choose the right statistical test**
   - Independent vs. Dependent samples
   - Parametric vs. Non-parametric methods

3. **Basic statistical tests for repeated measures**
   - Repeated Measures ANOVA (RM-ANOVA)
   - Friedman Test

4. **Beyond traditional tests: more flexible models**
   - GLM (Generalized Linear Model)
   - GLMM (Generalized Linear Mixed Model)
   - GEE (Generalized Estimating Equations)

➤ Each of these will be explored in more detail in the following sections.  

# Introduction: Why Classical Methods Are Not Enough

![scenario](https://drive.google.com/uc?export=view&id=1X6qZyy9vh0qcEsH7xky26uTv3gFq73tO)

In human-centered research, it is common to collect:
- Weekly stress ratings  
- Daily movement intensity  
- Physiological signals across tasks  

This results in **repeated measures data**, where each participant contributes multiple, often correlated observations.

As we work with human data, we naturally encounter repeated measures — and instead of treating each measurement as independent, we must **account for the within-subject structure** of the data.

This design allows us to:
- Detect variables that **significantly vary across conditions or over time**
- **Control for subject-specific effects** such as baseline differences

### Types of repeated measures scenarios:

➤ Based on conditions (e.g., labels or experimental groups):
- **Discrete / Binary labels**  
  - Stressed vs. Not-stressed
- **Continuous outcomes**  
  - Engagement score, HRV

➤ Based on time (temporal comparisons):
- Week 1, Week 2, Week 3  
- Morning vs. Evening  
- Before vs. After an intervention

### Within-subject and Between-subject Correlation

- **Within-subject correlation example:** A participant’s stress level this week is likely related to last week’s.
- **Between-subject correlation example:** Participants in the same condition or group (e.g., high-engagement group) may exhibit similar patterns.

To address these issues, we must adopt **statistical tests and models designed for repeated or clustered data** that can handle correlation and subject-level variability.

## 📊 Exploratory Data Analysis (EDA)

Before conducting any repeated measures analysis, let's start by **loading the dataset** and **exploring its basic structure**.

### Dataset Overview

This synthetic dataset contains weekly usage time (in minutes) of a digital journaling app, collected over 4 weeks from 46 participants. Participants are also categorized by:

- **StressLevel**: Low / Moderate / High
- **JournalStyle**: Reflective / Expressive
- **Gender**: Male / Female

### Initial Exploration Steps

1. Load the dataset `df1` and `df2`
2. Check for missing values
3. Inspect summary statistics
4. Visualize usage time by week and stress group

In [ ]:
import pandas as pd

# Load the practice dataset

df1 = pd.read_csv("/content/IoTLab07/data1.csv")
df2 = pd.read_csv("/content/IoTLab07/data2.csv")

In [ ]:
df1.head()

,UID,Gender,StressLevel,JournalStyle,W1_usage_time,W2_usage_time,W3_usage_time,W4_usage_time
0,P001,Male,High,Expressive,62.078298,49.610752,60.008773,56.779368
1,P002,Female,Moderate,Reflective,48.773021,54.545178,60.837781,59.773573
2,P003,Male,High,Expressive,63.972606,57.029088,75.233778,49.477655
3,P004,Male,High,Reflective,47.608170,NaN,NaN,NaN
4,P005,Male,Low,Reflective,65.421195,60.114002,56.793501,51.029677


In [ ]:
df2.head()

,UID,Gender,StressLevel,JournalStyle,W1_usage_time,W2_usage_time,W3_usage_time,W4_usage_time
0,P001,Male,Moderate,Reflective,18.770724,120.404857,52.669828,36.517702
1,P002,Male,High,Reflective,6.784995,6.783852,2.393551,NaN
2,P003,Female,Moderate,Reflective,36.763286,49.250002,NaN,140.142299
3,P004,Female,Moderate,Expressive,71.457182,9.547505,8.027160,8.104457
4,P005,Female,Low,Reflective,14.510149,NaN,22.621483,13.768920


In [ ]:
def check_missing_data(df, name):
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    print(f'{name} missing data:\n{missing}\n')

check_missing_data(df1, 'df1')
check_missing_data(df2, 'df2')

df1 missing data:
W1_usage_time     7
W2_usage_time     9
W3_usage_time    10
W4_usage_time     8
dtype: int64

df2 missing data:
W1_usage_time    5
W2_usage_time    6
W3_usage_time    7
W4_usage_time    4
dtype: int64



In [ ]:
# Summary statistics
df1.describe().T

,count,mean,std,min,25%,50%,75%,max
W1_usage_time,39.0,58.558167,7.369550,44.341659,52.259828,59.260655,64.800955,71.406533
W2_usage_time,37.0,58.369523,7.023481,42.698871,54.545178,59.034376,60.481192,72.144364
W3_usage_time,36.0,57.851152,7.391793,44.245951,53.558537,57.354909,62.630776,75.233778
W4_usage_time,38.0,56.061597,6.241861,39.958972,52.821601,54.874271,60.056455,66.887077


In [ ]:
# Summary statistics
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
W1_usage_time,41.0,35.680932,34.383818,0.279056,10.347470,24.357388,57.215710,131.280395
W2_usage_time,40.0,37.398763,38.516676,1.503379,8.608996,27.116387,51.276378,173.365854
W3_usage_time,39.0,44.992463,43.931186,1.029911,14.381242,27.506371,63.666270,169.761479
W4_usage_time,42.0,35.080856,35.269323,0.669078,9.157857,25.218111,51.661151,142.711926


The `melt()` function in pandas is used to reshape wide-format data into long format. This is useful when you need to analyze repeated measures data.

In [ ]:
def convert_to_long(df, value_vars):
    df_long = df.melt(
        id_vars=['UID', 'Gender', 'StressLevel', 'JournalStyle'],
        value_vars=value_vars,
        var_name='Week',
        value_name='UsageTime'
    )
    df_long['Week'] = df_long['Week'].str.extract(r'(W\d)')
    return df_long

usage_cols = ['W1_usage_time', 'W2_usage_time', 'W3_usage_time', 'W4_usage_time']
df1_long = convert_to_long(df1, usage_cols)
df2_long = convert_to_long(df2, usage_cols)

In [ ]:
# Randomly preview 5 rows to check long format structure
df1_long.sample(5)

,UID,Gender,StressLevel,JournalStyle,Week,UsageTime
19,P020,Male,Low,Reflective,W1,63.894520
26,P027,Female,Moderate,Reflective,W1,59.407006
11,P012,Male,High,Reflective,W1,44.341659
81,P036,Male,Low,Expressive,W2,60.146379
125,P034,Female,Moderate,Reflective,W3,49.443720


In [ ]:
import plotly.express as px

# JournalStyle by Week
fig = px.box(
    df1_long,
    x="Week",
    y="UsageTime",
    color="JournalStyle",
    title="Weekly Usage Time by Journal Style (Data 1)",
    labels={"UsageTime": "Usage Time (minutes)"},
    category_orders={"Week": ["W1", "W2", "W3", "W4"], "JournalStyle": ["Reflective", "Expressive"]}
)
fig.update_layout(boxmode='group', legend_title_text='Journal Style')
fig.show()


In [ ]:
fig = px.box(
    df2_long,
    x="Week",
    y="UsageTime",
    color="JournalStyle",
    title="Weekly Usage Time by Journal Style (Data 2)",
    labels={"UsageTime": "Usage Time (minutes)"},
    category_orders={"Week": ["W1", "W2", "W3", "W4"], "JournalStyle": ["Reflective", "Expressive"]}
)
fig.update_layout(boxmode='group', legend_title_text='Journal Style')
fig.show()


# Parametric vs. Non-parametric Approaches

Human-generated data — such as survey scores or biometric signals — often **do not follow a normal distribution**.

Using parametric tests like **t-tests or ANOVA** without checking distributional assumptions can lead to **distorted results**.  
In such cases, **non-parametric methods** are more appropriate.

### Summary:

|               |             | Parametric                   | Non-parametric             |
|---------------|-------------|------------------------------|----------------------------|
| Independent   | 2           | Independent t-test           | Mann-Whitney U test        |
| Independent   | ≥3          | One-way ANOVA                | Kruskal-Wallis test        |
| Dependent     | 2           | Paired t-test                | Wilcoxon signed-rank test  |
| Dependent     | ≥3          | Repeated Measures ANOVA      | Friedman test              |

Always assess whether your data meets the assumptions of parametric tests (normality, equal variance, etc.).

## Normality Test

### **How to Check Normality?**

There are several methods to check for normality in a dataset.  
Here are three widely used approaches:

| **Method**  | **When to Use?** |
|------------------|-------------|
| **Shapiro-Wilk Test**  | Widely used for small to medium samples |
| **Kolmogorov-Smirnov Test**  | Suitable for larger samples; tests skewness and kurtosis |
| **Q–Q Plots**  | Visual assessment by comparing quantiles of data against a normal distribution |


### **What if Normality is Violated?**
- **Apply transformations** (e.g., log, square root) to normalize data.
- **Use non-parametric tests** instead.

### **Exercise: Run Normality Test and Select the Appropriate Statistical Test**
1️⃣ Run a **Shapiro-Wilk test** on `df1` and `df2`.  
2️⃣ Based on the normality results:
   - If normal → Use **parametric tests**.
   - If non-normal → Use **non-parametric tests**.

3️⃣ **Write a short conclusion**: Which test is appropriate and why?

In [ ]:
from scipy.stats import shapiro

# Function to perform Shapiro-Wilk test for each week
def shapiro_test_per_week(df, group_col, value_col):
    results = []
    for week in df[group_col].unique():
        data = df[df[group_col] == week][value_col].dropna()

        # Shapiro-Wilk Test
        shapiro_stat, shapiro_p = shapiro(data)

        results.append({
            'Week': week,
            'Shapiro-Wilk Statistic': shapiro_stat,
            'p-value': shapiro_p
        })

    return pd.DataFrame(results)

# Run Shapiro-Wilk test
shapiro_results_df1 = shapiro_test_per_week(df1_long, 'Week', 'UsageTime')
shapiro_results_df2 = shapiro_test_per_week(df2_long, 'Week', 'UsageTime')

In [ ]:
# All weeks have p-value > 0.05, indicating that the data is approximately normally distributed for each week.
shapiro_results_df1

,Week,Shapiro-Wilk Statistic,p-value
0,W1,0.949838,0.081147
1,W2,0.979304,0.707394
2,W3,0.979190,0.718062
3,W4,0.969389,0.375530


In [ ]:
# All weeks have p-value < 0.05, indicating that the data isnt't approximately normally distributed for each week.
shapiro_results_df2

,Week,Shapiro-Wilk Statistic,p-value
0,W1,0.862667,0.000156
1,W2,0.822307,0.000020
2,W3,0.846452,0.000089
3,W4,0.814714,0.000009


Therefore, `df1` follows an approximately normal distribution, but `df2` does not.

## **One Way Repeated Measures ANOVA (RM-ANOVA)**

- Parametric test
- Assumptions [(Ref)](https://pmc.ncbi.nlm.nih.gov/articles/PMC10231988/pdf/jci-133-171058.pdf):
  - Continuous dependent variable is approximately normally distributed
  - Categorical independent variable has three or more levels
  - No outliers in any of the repeated measurements
  - Sphericity (constant variance across time points)
  - Balanced number of repeated measurements for each experimental unit
- Typically used when comparing **3 or more time points** for **the same subjects**

### Example Scenario
- Measuring usage time across Week 1, Week 2, Week 3, and Week 4 for each subject.

### Limitations
- Cannot handle missing data
- Assumes equal variance of differences (sphericity)

In [ ]:
# Step 1: Remove participants with missing values
# Count missing values per participant
missing_counts = df1_long.groupby("UID")["UsageTime"].apply(lambda x: x.isnull().sum())

# Keep only participants with NO missing values
valid_subjects = missing_counts[missing_counts == 0].index
df1_no_missing = df1_long[df1_long["UID"].isin(valid_subjects)]
print(f"{len(valid_subjects)} participants' data remained after removing missing data.\n")

22 participants' data remained after removing missing data.



For **handling missing data**, we could apply **imputation techniques** (such as mean or median imputation), but in this lecture, we'll skip it due to the focus on understanding statistical models.

In [ ]:
import pingouin as pg

# Step 2: Run Mauchly’s Test for Sphericity
mauchly_test = pg.sphericity(df1_no_missing, dv="UsageTime", subject="UID", within="Week")

# Display Mauchly’s Test result
print("Mauchly’s Test for Sphericity:")
print(mauchly_test)

# Interpretation: If p > 0.05 → Sphericity assumption is not violated
if mauchly_test.pval > 0.05:
    print("Sphericity assumption is met.")
else:
    print("Sphericity assumption is violated.")

Mauchly’s Test for Sphericity:
SpherResults(spher=True, W=np.float64(0.9629503983271704), chi2=np.float64(0.7445804703022397), dof=5, pval=np.float64(0.9804561850917628))
Sphericity assumption is met.


In [ ]:
# Step 3: Run One-way RM-ANOVA
rm_anova_results = pg.rm_anova(data=df1_no_missing, dv='UsageTime', within='Week', subject='UID', detailed=True)

# Step 4: Display results
print(rm_anova_results)

  Source           SS  DF          MS        F     p-unc      ng2      eps
0   Week   339.995602   3  113.331867  3.92515  0.012397  0.07608  0.97739
1  Error  1819.015024  63   28.873254      NaN       NaN      NaN      NaN


Since the **p-value is smaller than 0.05**, there is **statistically difference** in usage time across weeks.

➤ Then we can perform **post-hoc tests** to further analyze which specific weeks differ from each other.

In [ ]:
# Step 5: Post-hoc tests corrected for multiple comparisons
posthoc = pg.pairwise_tests(data=df1_no_missing, dv='UsageTime', within='Week', subject='UID',
                             parametric=True, padjust='fdr_bh', effsize='hedges')

# Step 6: Display results
pg.print_table(posthoc, floatfmt='.3f')


POST HOC TESTS

Contrast    A    B    Paired    Parametric         T     dof  alternative      p-unc    p-corr  p-adjust      BF10    hedges
----------  ---  ---  --------  ------------  ------  ------  -------------  -------  --------  ----------  ------  --------
Week        W1   W2   True      True           0.530  21.000  two-sided        0.602     0.749  fdr_bh       0.253     0.111
Week        W1   W3   True      True           0.497  21.000  two-sided        0.624     0.749  fdr_bh       0.249     0.112
Week        W1   W4   True      True           2.948  21.000  two-sided        0.008     0.029  fdr_bh       6.210     0.682
Week        W2   W3   True      True          -0.006  21.000  two-sided        0.995     0.995  fdr_bh       0.223    -0.001
Week        W2   W4   True      True           2.844  21.000  two-sided        0.010     0.029  fdr_bh       5.098     0.612
Week        W3   W4   True      True           2.593  21.000  two-sided        0.017     0.034  fdr_bh      

## **Friedman test**
- Non-parametric alternative to RM-ANOVA
- No assumption of normality
- Used when data are ordinal or not normally distributed

### Example Scenario
- Comparing Likert-scale stress scores over 3 sessions (ordinal data)

### When to use?
- Small sample sizes
- Violated normality assumption

In [ ]:
from scipy.stats import friedmanchisquare

# Step 1: Remove participants with missing values
# Count missing values per participant
missing_counts = df2_long.groupby("UID")["UsageTime"].apply(lambda x: x.isnull().sum())

# Keep only participants with NO missing values
valid_subjects = missing_counts[missing_counts == 0].index
df2_no_missing = df2_long[df2_long["UID"].isin(valid_subjects)]
print(f"{len(valid_subjects)} participants' data remained after removing missing data.\n")

# Step 2: Prepare data for Friedman Test
# Pivot the data to wide format
df2_no_missing_wide = df2_no_missing.pivot(index='UID', columns='Week', values='UsageTime')

# Step 3: Run Friedman Test
friedman_results = friedmanchisquare(
    df2_no_missing_wide['W1'],
    df2_no_missing_wide['W2'],
    df2_no_missing_wide['W3'],
    df2_no_missing_wide['W4']
)

# Step 4: Display results
print(friedman_results)

27 participants' data remained after removing missing data.

FriedmanchisquareResult(statistic=np.float64(0.466666666666697), pvalue=np.float64(0.9261511402759927))


Since the **p-value is greater than 0.05**, there is **no statistically significant difference** in usage time across weeks.

# Beyond Traditional Tests: Model-Based Approaches

Basic repeated measures tests (like RM-ANOVA and Friedman) are useful, but they have important limitations:

- Cannot handle **missing data** gracefully
- Require **equal time intervals** or **balanced designs**
- Cannot easily model **nonlinear trends** or **individual variation**

To overcome these issues, we use **model-based approaches**:

### Advanced Models:

| Model | Description |
|-------|-------------|
| **GLM** (Generalized Linear Model) | Extends regression to handle various outcome types (e.g., binary, count) |
| **GLMM** (Generalized Linear Mixed Model) | Adds random effects to GLM → accounts for subject-specific variability |
| **GEE** (Generalized Estimating Equation) | Population-level model that handles correlation without requiring random effects |

These models allow:
- Flexibility in outcome distributions (not just normal)
- Modeling of time trends and subject-level random effects
- Robust handling of correlated and incomplete data

# Generalized Linear Model (GLM)

### **Understanding Probability Distributions**
Before choosing a statistical test, we must understand how **data is distributed**. The type of distribution determines which Generalized Linear Model (GLM) we should use.

#### **Common Probability Distributions**
| **Distribution**  | **Equation** | **Use Case** | **Example in Sensor Data** |
|------------------|-------------|-------------|-----------------------------|
| **Normal**      | $ f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x - \mu)^2}{2\sigma^2}} $ | Symmetric data | Heart rate variability at baseline |
| **Poisson**     | $ P(X=k) = \frac{\lambda^k e^{-\lambda}}{k!} $ | Count data | Steps per hour |
| **Gamma**       | $ f(x) = \frac{x^{k-1}e^{-x/\theta}}{\theta^k \Gamma(k)} $ | Skewed continuous data | RR intervals |
| **Exponential** | $ f(x) = \lambda e^{-\lambda x} $ | Time until event | Time between detected heartbeats |

Below is a visualization of each distribution using the given parameters.
- Normal: `μ=0, σ=1`
- Poisson: `λ=5`
- Gamma: `k=2, θ=2`
- Exponential: `λ=1`

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as sp
import scipy.stats as stats

# Set the size of the dataset
size = 10000

# Generate random data from different distributions
np.random.seed(42)  # Set seed for reproducibility
normal_data = np.random.normal(loc=0, scale=1, size=size)
poisson_data = np.random.poisson(lam=5, size=size)
gamma_data = np.random.gamma(shape=2, scale=2, size=size)
exponential_data = np.random.exponential(scale=1, size=size)

# Create subplot figure
fig = sp.make_subplots(rows=1, cols=4, subplot_titles=["Normal", "Poisson", "Gamma", "Exponential"])

# Define distributions and colors
distributions = [(normal_data, 'teal', 'Normal'),
                 (poisson_data, 'purple', 'Poisson'),
                 (gamma_data, 'blue', 'Gamma'),
                 (exponential_data, 'green', 'Exponential')]

# Add histogram and density estimation for each distribution
for i, (data, color, name) in enumerate(distributions):
    # Add histogram
    fig.add_trace(go.Histogram(x=data, nbinsx=50, histnorm='probability density', marker_color=color, opacity=0.6), row=1, col=i+1)

    # Apply KDE only for continuous distributions
    if name != "Poisson":
        kde_x = np.linspace(min(data), max(data), 100)
        kde = stats.gaussian_kde(data)
        kde_y = kde(kde_x)

        # Add density line
        fig.add_trace(go.Scatter(x=kde_x, y=kde_y, mode='lines', line=dict(color=color, width=2), name=f'{name} Density'), row=1, col=i+1)

# Configure layout
fig.update_layout(title='Different Types of Distributions', template='plotly_white', showlegend=False)

# Show plot
fig.show()

### **Mathematical Formulation of GLM**
A **GLM** consists of three components:

1. **Random Component**: Defines the **distribution** of the response variable $ Y $.
2. **Systematic Component**: A linear predictor $ \eta $ given by:
   $$
   \eta = X\beta
   $$
3. **Link Function**: Transforms the expectation of $ Y $ to match $ \eta $:
   $$
   g(E(Y)) = X\beta
   $$

### **Common GLM Models**
Each type of GLM uses a different **link function** and **distribution**:

| **Model Type**  | **Response Variable** | **Link Function** | **Example Use Case** |
|----------------|---------------------|-----------------|----------------|
| **Logistic Regression**  | Binary (0/1) | $ \log \frac{p}{1-p} $ | Stress level (High vs. Low) |
| **Poisson Regression**  | Count Data | $ \log(\lambda) $ | Stress events per hour |
| **Gamma Regression**  | Continuous (Skewed) | $ 1/\mu $ or log | Time between HRV peaks |

By choosing an appropriate probability distribution and link function, GLM enables flexible modeling for various types of data.

### 🚨**Limitation in repeated measures context:**
GLM assumes **independent observations**.
However, repeated measures data violates this assumption → standard errors become incorrect, leading to invalid inference.

➡️ That’s why we need models like **GLMM** and **GEE** that explicitly account for within-subject correlation.

## Extension of GLM
### **Generalized Linear Mixed Model (GLMM)**
- GLM + **random effects**
- Captures **subject-level variability**
- Models individual-specific trajectories
- Useful for:
  - Repeated observations per subject
  - Nested or hierarchical data

### **Generating Estimation Equation (GEE)**
- Also handles repeated measures  
- Focuses on **population-averaged effects** (i.e., average change across all individuals)  
- Does **not model random effects** explicitly  
- Uses a **working correlation matrix** (e.g., exchangeable, autoregressive AR(1)) to account for within-subject correlation

GEE is especially useful when the research goal is to estimate **population-level trends**, rather than tracking individual-specific patterns.  
It is commonly used in epidemiological and public health research, where **subject-level variation is treated as a nuisance**.

However, GEE does not model subject-level variation, so it may not be ideal if individual trajectories or heterogeneity are of interest.

**When is GEE recommended?**

According to [Neuhaus et al., 2006](https://www.sciencedirect.com/science/article/pii/S0306460306001018?ref=pdf_download&fr=RR-2&rr=923b19d5fa40ea15):
> GEE is appropriate **when the number of participants is at least 30**, and each participant has **3 to 5 repeated observations**.

This is because GEE relies on **large sample theory (asymptotic properties)** for its parameter estimates to be reliable.

So if your study design satisfies these criteria, and your interest lies in **population-level effects**, then **GEE is a robust and interpretable choice** for repeated measures analysis.

### 📌 GLMM vs GEE

| Feature | GLMM | GEE |
|--------|------|-----|
| Random effects | ✅ | ❌ |
| Subject-level inference | ✅ | ❌ |
| Population-level inference | ✅ | ✅ |
| Handles missing data | ✅ (better) | ✅ |
| Interpretation | Subject-specific | Population-averaged |

In practice, GLMM might tell you how much **a specific person** improves after intervention, while GEE would tell you **on average** how the population changes.

In [ ]:
import statsmodels.formula.api as smf

# ---------------------------- #
# Run GLMM (Generalized Linear Mixed Model)
# ---------------------------- #
glmm_model = smf.mixedlm("UsageTime ~ Week * JournalStyle", df2_long, groups=df2_long["UID"], missing='drop')
glmm_results = glmm_model.fit()

# Display GLMM results
print(glmm_results.summary())

# Interpretation of GLMM results:
# - A positive coefficient for "Week" indicates that usage time increases over time.
# - Significant interaction effects between "JournalStyle" and "Week * JournalStyle" suggest that journal style may influence weekly usage patterns.
# - Random effects (variability in intercepts) explain differences between participants.

                      Mixed Linear Model Regression Results
Model:                     MixedLM          Dependent Variable:          UsageTime
No. Observations:          162              Method:                      REML     
No. Groups:                46               Scale:                       1243.1968
Min. group size:           2                Log-Likelihood:              -786.1833
Max. group size:           4                Converged:                   Yes      
Mean group size:           3.5                                                    
----------------------------------------------------------------------------------
                                       Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
----------------------------------------------------------------------------------
Intercept                              37.305    9.281  4.020 0.000  19.115 55.494
Week[T.W2]                            -16.547   12.574 -1.316 0.188 -41.192  8.098
Week[T.W3]                 

In [ ]:
# Recap: Usage Time by Week
fig = px.box(
    df2_long,
    x="Week",
    y="UsageTime",
    title="Weekly Usage Time (Data 2)",
    labels={"UsageTime": "Usage Time (minutes)"},
    category_orders={"Week": ["W1", "W2", "W3", "W4"]}
)
fig.update_layout(boxmode='group', legend_title_text='Journal Style')
fig.show()

So far, we have examined how **Week** and **JournalStyle** affect **Usage Time** while accounting for participant-level random effects.  
However, we can further extend our model by incorporating **Gender** into the interaction term to investigate whether usage trends differ by gender.

Below, we modify our GLMM to include **Week × JournalStyle × Gender** interactions:


In [ ]:
# Run GLMM with Week, JournalStyle, and Gender
glmm_model = smf.mixedlm("StressLevel ~ Week * JournalStyle * Gender",
                         df2_long, groups=df2_long["UID"], re_formula="Week", missing='drop')
# Fit model
glmm_results = glmm_model.fit()

# Display results
print(glmm_results.summary())

                               Mixed Linear Model Regression Results
Model:                            MixedLM               Dependent Variable:               UsageTime
No. Observations:                 162                   Method:                           REML     
No. Groups:                       46                    Scale:                            865.4760 
Min. group size:                  2                     Log-Likelihood:                   -752.0388
Max. group size:                  4                     Converged:                        Yes      
Mean group size:                  3.5                                                              
---------------------------------------------------------------------------------------------------
                                                      Coef.   Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------------------------------------
Intercept                      

In [ ]:
import statsmodels.api as sm

# Define correlation structure
independent_corr = sm.cov_struct.Independence()
exchangeable_corr = sm.cov_struct.Exchangeable()
ar1_corr = sm.cov_struct.Autoregressive(1)

# Run GEE model
gee_model = sm.GEE.from_formula("UsageTime ~ Week * JournalStyle",
                                groups=df2_long["UID"], cov_struct=exchangeable_corr,  # Use exchangeable correlation
                                data=df2_long)

# Fit model
gee_results = gee_model.fit()

# Display results
print(gee_results.summary())

# Interpretation of GEE results:
# - GEE explains the population-averaged effect, so there are no random effects to explain individual differences.
# - If the "Working correlation" value is ρ=0.3 or higher, it indicates significant correlation between measurements over time.

                               GEE Regression Results                              
Dep. Variable:                   UsageTime   No. Observations:                  162
Model:                                 GEE   No. clusters:                       46
Method:                        Generalized   Min. cluster size:                   2
                      Estimating Equations   Max. cluster size:                   4
Family:                           Gaussian   Mean cluster size:                 3.5
Dependence structure:         Exchangeable   Num. iterations:                     6
Date:                     Sat, 22 Mar 2025   Scale:                        1377.528
Covariance type:                    robust   Time:                         10:24:41
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                       

/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/cov_struct.py:796: FutureWarning:

grid=True will become default in a future version



We can incorporate **Gender** into the interaction term in GEE as well, but we will skip it.

# Homework #6: Anlayzing repeated measures data

*   3 points towards your final score
*   Must submit a single colab file to KLMS by next Wednesday 23:59:59 (April 9th)

In this homework, you will explore how different smartphone **placements** influence motion intensity, as measured by **acceleration variance** during a fixed physical activity (e.g., walking).

> ⚠️ **Note**: This dataset is **synthetic**, and was generated for instructional purposes by the TA.

This synthetic dataset follows a **repeated measures design**, where the same participant is observed under four different placement conditions.

Each participant performs the same activity while placing the smartphone in the following locations:

- `PantsPocket` (inside pants pocket)  
- `Hand` (held in hand)  
- `Bag` (inside a backpack or shoulder bag)  
- `JacketPocket` (inside jacket or coat pocket)

Additionally, participants are categorized into two groups based on the weight of the smartphone device they used:  
**Light** vs. **Heavy**. This variable is provided as `DeviceWeightGroup`, and can be used to investigate between-subject differences.


Your task is to analyze it through the following steps:

### (1) Exploratory Data Analysis (0.6 pt)

- (a) Check for any missing values and summarize descriptive statistics. (0.2 pt)
- (b) Change your data frame from wide to long. (0.2 pt)
- (c) Visualize **acceleration variance across phone placements**, grouped by device weight category (`Light` vs. `Heavy`). (0.2 pt)

In [ ]:
# Load data
df = pd.read_csv("/content/IoTLab07/data_hw.csv")

# Write your code here

# -----------------------------------------------
# (1) Exploratory Data Analysis (0.6 pt)
# -----------------------------------------------

# (a) Check missing data and describe overall statistics (0.2 pt)
print("Missing value summary:")
print(df.isnull().sum())

print("Overall descriptive statistics:")
df.describe().T

Missing value summary:
UID                  0
DeviceWeightGroup    0
PantsPocket          0
Hand                 0
Bag                  0
JacketPocket         0
dtype: int64
Overall descriptive statistics:


,count,mean,std,min,25%,50%,75%,max
PantsPocket,30.0,0.867057,0.732796,0.065668,0.228946,0.695626,1.382978,2.549435
Hand,30.0,1.696988,1.724797,0.011075,0.324572,0.864619,2.928018,5.610189
Bag,30.0,3.703329,3.913116,0.190255,0.941143,2.645227,4.270497,17.336585
JacketPocket,30.0,2.982690,2.879769,0.077243,1.057570,1.658716,4.297159,10.478421


In [ ]:
# (b) Change your data frame from wide to long. (0.2 pt)
# Hint: Use `melt()` function.

# Write your code here.

df_long = pd.melt(
    df,
    id_vars=["UID", "DeviceWeightGroup"],
    value_vars=["PantsPocket", "Hand", "Bag", "JacketPocket"],
    var_name="PhonePlacement",
    value_name="AccelVariance"
)

# Preview
print(f"shape: {df_long.shape}")
df_long.head()

shape: (120, 4)


,UID,DeviceWeightGroup,PhonePlacement,AccelVariance
0,P001,Light,PantsPocket,0.202611
1,P002,Heavy,PantsPocket,0.344223
2,P003,Light,PantsPocket,0.456277
3,P004,Light,PantsPocket,0.722029
4,P005,Light,PantsPocket,0.186961


In [ ]:
# (c) Visualize acceleration variance across phone placements,
#     grouped by device weight category (`Light` vs. `Heavy`). (0.2 pt)
# Hint: Visualize boxplot

# Write your code here.

fig = px.box(
    df_long,
    x="PhonePlacement",
    y="AccelVariance",
    color="DeviceWeightGroup",
    title="AccelVariance by Phone Placement (Grouped by Device Weight)",
    labels={"AccelVariance": "Acceleration Variance"},
    category_orders={"PhonePlacement": ["PantsPocket", "Hand", "Bag", "JacketPocket"]}
)

fig.update_layout(
    boxmode='group',
    legend_title_text='Device Weight Group'
)
fig.show()


### (2) Normality Test (1.0 pt)

- (a) Define your null (H₀) and alternative (H₁) hypotheses. (0.2 pt)  
- (b) Perform a normality test on the data. (0.2 pt)  
- (c) Interpret the result. Can you reject H₀? (0.2 pt)  
  Example:  
  *Since p-value is (     ), we (can/cannot) reject the null hypothesis. The data (are/aren’t) approximately normally distributed.*

- (d) Apply a transformation of your choice: (0.2 pt)  
    - `log(x + 1)`  
    - `ln(x + 1)`  
    - `sqrt(x)`  
    - `1 / x`

- (e) Re-run the normality test on the transformed data and nterpret the new result. (0.2 pt)

In [ ]:
# -----------------------------------------------
# (2) Normality Test (1.0 pt)
# -----------------------------------------------

# (a) Define your null and alternative hypotheses. (0.2 pt)

print("H0: ") # Write your null hypothesis
print("H1: ") # Write your alternative hypothesis

# Answer
print("H0:: The data are drawn from a normal distribution.")
print("H1: The data are not drawn from a normal distribution.")

H0: 
H1: 
H0:: The data are drawn from a normal distribution.
H1: The data are not drawn from a normal distribution.


In [ ]:
# Write your code here

# (b) Normality test (Shapiro-Wilk) (0.2 pt)
print("Shapiro-Wilk test (raw values):")
stat, p = shapiro(df_long["AccelVariance"])
print(f"p-value = {p:.5f}")

# (c) Interpret the result (0.2 pt)
print("Since p-value < 0.05, we reject the null hypothesis.")
print("The data are not approximately normally distributed.")

# (d) Apply transformation (0.2 pt)
df_long["AccelVariance_log"] = np.log(df_long["AccelVariance"] + 1)

# (e) Re-test for normality and interpret the result (0.2 pt)
print("\nShapiro-Wilk test (log-transformed):")
stat_log, p_log = shapiro(df_long["AccelVariance_log"])
print(f"p-value = {p_log:.5f}")

print("Since p-value < 0.05, we still reject the null hypothesis.")
print("Even after log transformation, the data are not approximately normally distributed.")

Shapiro-Wilk test (raw values):
p-value = 0.00000
Since p-value < 0.05, we reject the null hypothesis.
The data are not approximately normally distributed.

Shapiro-Wilk test (log-transformed):
p-value = 0.00014
Since p-value < 0.05, we still reject the null hypothesis.
Even after log transformation, the data are not approximately normally distributed.


### (3) Statistical Analysis (1.4 pt)

#### (3-1) Choose and Run a Traditional Test (0.6 pt)

- (a) Based on your normality test results:  
  - If the data are approximately normal → run **Repeated Measures ANOVA**  
  - If the data are not normal → run the **Friedman test**  
- Report the test results. Are there any significant differences between conditions? (0.3 pt)

- (b) Check whether there are significant differences between conditions.  
  If so, perform a post-hoc test and interpret the results. (0.3 pt)

In [ ]:
# -----------------------------------------------
# (3-1) Traditional Test (0.6 pt)
# -----------------------------------------------

# Write your code here

In [ ]:
# -----------------------------------------------
# (3-1) Traditional Test (0.6 pt)
# -----------------------------------------------

# (a) Apply appropriate test

# Pivot data for Friedman
df_friedman = df_long.pivot(index="UID", columns="PhonePlacement", values="AccelVariance").dropna()

# Run Friedman test
friedman_stat, friedman_p = friedmanchisquare(
    df_friedman["PantsPocket"],
    df_friedman["Hand"],
    df_friedman["Bag"],
    df_friedman["JacketPocket"]
)
print(f"\nFriedman Test: stat={friedman_stat:.2f}, p={friedman_p:.5f}")


Friedman Test: stat=16.16, p=0.00105


In [ ]:
# Post-hoc test
posthoc = pg.pairwise_tests(
    data=df_long,
    dv="AccelVariance",
    within="PhonePlacement",
    subject="UID",
    parametric=False,
    padjust="fdr_bh"
)
print("\nPost-hoc comparisons:")
posthoc


Post-hoc comparisons:


,Contrast,A,B,Paired,Parametric,W-val,alternative,p-unc,p-corr,p-adjust,hedges
0,PhonePlacement,Bag,Hand,True,False,113.0,two-sided,0.012834,0.025667,fdr_bh,0.654888
1,PhonePlacement,Bag,JacketPocket,True,False,182.0,two-sided,0.308521,0.308521,fdr_bh,0.207037
2,PhonePlacement,Bag,PantsPocket,True,False,53.0,two-sided,0.000079,0.000475,fdr_bh,0.994440
3,PhonePlacement,Hand,JacketPocket,True,False,156.0,two-sided,0.119077,0.142892,fdr_bh,-0.534632
4,PhonePlacement,Hand,PantsPocket,True,False,131.0,two-sided,0.036435,0.054652,fdr_bh,0.618170
5,PhonePlacement,JacketPocket,PantsPocket,True,False,68.0,two-sided,0.000380,0.001140,fdr_bh,0.993794


#### (3-2) Apply Model-Based Approaches (0.8 pt)

- (a) Fit a **GLMM** and interpret the result. (0.3 pt)  
- (b) Fit a **GEE** with AR(1) correlation and interpet the result. (0.3 pt)  
- (c) Compare GLMM and GEE results. (0.2 pt)

In [ ]:
# -----------------------------------------------
# (3-2) Model-Based Approaches (0.6 pt)
# -----------------------------------------------

# Hint: You should include PhonePlacement and DeviceWeightGroup as fixed effects,
#       and treat `UID` as a random intercept to account for repeated measurements from the same participant.


In [ ]:
# -----------------------------------------------
# (3-b) Model-Based Approaches (0.6 pt)
# -----------------------------------------------

# GLMM (random intercept for UID)
glmm_model = smf.mixedlm("AccelVariance ~ PhonePlacement + DeviceWeightGroup", data=df_long, groups="UID")
glmm_result = glmm_model.fit()
print("\nGLMM Summary:")
print(glmm_result.summary())


GLMM Summary:
                  Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     AccelVariance
No. Observations:       120         Method:                 REML         
No. Groups:             30          Scale:                  6.6685       
Min. group size:        4           Log-Likelihood:         -282.1683    
Max. group size:        4           Converged:              Yes          
Mean group size:        4.0                                              
-------------------------------------------------------------------------
                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                       3.685    0.531  6.941 0.000  2.644  4.725
PhonePlacement[T.Hand]         -2.006    0.667 -3.009 0.003 -3.313 -0.700
PhonePlacement[T.JacketPocket] -0.721    0.667 -1.081 0.280 -2.027  0.586
PhonePlacement[T.PantsPocket]  -2.836    

In [ ]:
# GEE with AR(1)
gee_model = smf.gee("AccelVariance ~ PhonePlacement + DeviceWeightGroup", groups="UID",
                    data=df_long, cov_struct=sm.cov_struct.Autoregressive())
gee_result = gee_model.fit()
print("\nGEE Summary:")
print(gee_result.summary())


GEE Summary:
                               GEE Regression Results                              
Dep. Variable:               AccelVariance   No. Observations:                  120
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                   4
                      Estimating Equations   Max. cluster size:                   4
Family:                           Gaussian   Mean cluster size:                 4.0
Dependence structure:       Autoregressive   Num. iterations:                     4
Date:                     Sat, 22 Mar 2025   Scale:                           6.838
Covariance type:                    robust   Time:                         10:30:29
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                       

/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/cov_struct.py:796: FutureWarning:

grid=True will become default in a future version

